# 0. Library imports

In [1]:
# Step 4: authenticate
from google.cloud import bigquery
from google.oauth2 import service_account

# Step 5: pre-process data
import pandas as pd
import re
import zipfile
from urllib.request import urlopen
from io import BytesIO

# 1. Set up project in GCP
Using console.cloud.google.com, we set up a project called **tc-transit**. This project name will get referenced throughout our scripts.

# 2. Generate owner JSON key
Next, we created a service account and generated a JSON key with owner access to that service account (see [these instructions](https://cloud.google.com/iam/docs/creating-managing-service-account-keys#iam-service-account-keys-create-console)).

# 3. Upload JSON key to Colab 
We uploaded data and the JSON key from step 2 to the `tc-transit` folder containing this and the other scripts.

# 4. Authenticate

In [2]:
# Mount drive - needed for GBQ creds
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
root_path = '/content/drive/MyDrive/GitHub'

In [4]:
credentials = service_account.Credentials.from_service_account_file(
    f'{root_path}/tc-transit/tc-transit-81fcba2c3195.json', scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [67]:
# For running queries using BigQuery magic

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


# 5. Import data and pre-process

In [44]:
req = urlopen('https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/trans_stop_boardings_alightings/csv_trans_stop_boardings_alightings.zip')
zip_file = zipfile.ZipFile(BytesIO(req.read()))
tc_transit = pd.read_csv(zip_file.open('TransitStopsBoardingsAndAlightings2021.csv'), index_col = False)

In [50]:
# Clean up/normalize column names
tc_rnm = tc_transit.rename(columns = lambda x: re.sub(' |-', '_', x.lower().replace('%', '')))

# Create relations
routes = tc_rnm[['provider', 'route', 'dir', 'route_type', 'group_id', 'site_id', 'seq']].drop_duplicates()
sites = tc_rnm[['site_id', 'geo_node_name', 'latitude', 'longitude', 'x_coordinate', 'y_coordinate', 'city', 'downtown']].drop_duplicates()
trips = tc_rnm[['site_id', 'route', 'dir', 'schedule', 'trips', 'obs_trips', 'ons', 'offs', 'sampled', 'comment']].drop_duplicates()
trips['year'] = 2021

# Get rid of indexes before loading
routes.reset_index(drop = True, inplace = True)
sites.reset_index(drop = True, inplace = True)
trips.reset_index(drop = True, inplace = True)

#6. Push dataframes to persistent relations in GCP

Create a dataset called transit within the project (tc-transit). Upload the newly cleaned dataframes to the tc-transit.transit.routes, tc-transit.transit.sites, tc-transit.transit.trips tables. Both the dataset and tables are persistent, so this code only needs to be run once.

For help specifying the schema, see these instructions.

In [40]:
# Create transit dataset
transit = bigquery.Dataset('tc-transit.transit')
transit.location = 'US'
transit = client.create_dataset(transit)

# Configure job
job_config = bigquery.LoadJobConfig(
    autodetect = True,
    source_format = bigquery.SourceFormat.CSV)

# Load dfs
job = client.load_table_from_dataframe(routes, 'tc-transit.transit.routes', job_config = job_config)
job = client.load_table_from_dataframe(sites, 'tc-transit.transit.sites', job_config = job_config)
job = client.load_table_from_dataframe(trips, 'tc-transit.transit.trips', job_config = job_config)

# 7. Test that tables were successfully created

In [70]:
%%bigquery --project tc-transit tc645
SELECT * except(__index_level_0__)
FROM `transit.trips`
WHERE route = '645'
AND dir = 'East'
AND schedule = 'Weekday';

In [71]:
tc645.head()

,site_id,route,dir,schedule,trips,obs_trips,ons,offs,sampled,comment,year
0,52852.0,645,East,Weekday,14.0,14.0,20,1,100%,None,2021
1,41157.0,645,East,Weekday,22.0,22.0,28,4,100%,None,2021
2,7005.0,645,East,Weekday,23.0,23.0,40,8,100%,None,2021
3,52776.0,645,East,Weekday,5.0,5.0,0,0,100%,None,2021
4,52775.0,645,East,Weekday,5.0,5.0,0,0,100%,None,2021


In [64]:
t645_q = '''SELECT * except(__index_level_0__)
FROM `transit.trips`
WHERE route = '645'
AND dir = 'East'
AND schedule = 'Weekday';'''

t645 = client.query(t645_q).to_dataframe()

In [65]:
t645.head()

,site_id,route,dir,schedule,trips,obs_trips,ons,offs,sampled,comment,year,__index_level_0__
0,52852.0,645,East,Weekday,14.0,14.0,20,1,100%,None,2021,11750
1,41157.0,645,East,Weekday,22.0,22.0,28,4,100%,None,2021,11794
2,7005.0,645,East,Weekday,23.0,23.0,40,8,100%,None,2021,11823
3,52776.0,645,East,Weekday,5.0,5.0,0,0,100%,None,2021,11806
4,52775.0,645,East,Weekday,5.0,5.0,0,0,100%,None,2021,11807
